# [모듈 2.1] 환경 설정 및 데이터 준비

## 0. 기본 세이지 메이커 정보 로딩

In [6]:
import boto3
import sagemaker

import os
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()


## 1. 노트북 변수 설정
---

In [7]:
# 프로젝트 변수
project_prefix = 'fraud2scratch'
dataset_prefix = 'fraud2scratch/input'

# 데이타 위치 및 파일 경로 
data_dir = '../data'
base_preproc_input_dir = 'opt/ml/processing/input'
os.makedirs(base_preproc_input_dir, exist_ok=True)

train_data_path = f"{data_dir}/train.csv"
test_data_path = f"{data_dir}/test.csv"
dataset_path = "{}/dataset.csv".format(base_preproc_input_dir)    
s3_dataset_path = f"s3://{default_bucket}/{dataset_prefix}"

# 모델 패키지 그룹 이름
model_package_group_name = f"FraudScratchModelPackageGroupName"

# 2. 데이터 준비
---

## 데이터 세트 로컬 저장

In [8]:
def get_dataset(train_data_path, test_data_path):
    df_train = pd.read_csv(train_data_path, index_col=0)
    df_test = pd.read_csv(test_data_path, index_col=0)
    df_dataset = pd.concat([df_train, df_test], axis=0)
    print("df_train shape: ", df_train.shape)
    print("df_test shape: ", df_test.shape)

    df_dataset = df_dataset.reset_index() # label 컴럼이 index로 되어 있어서, 컬럼으로 추출
    df_dataset.to_csv(dataset_path, index=None)
    print(f"{dataset_path} is saved")
    
    dataset_df = pd.read_csv(dataset_path)
    print("df_dataset shape: ", dataset_df.shape) # fraud 추가

get_dataset(train_data_path, test_data_path)    

df_train shape:  (16000, 45)
df_test shape:  (4000, 45)
opt/ml/processing/input/dataset.csv is saved
df_dataset shape:  (20000, 46)


## S3에 데이터 세트 업로딩

이제 데이터를 디폴트 버킷으로 업로드합니다. `input_data_uri` 변수를 통해 데이터셋의 위치를 저장하였습니다.

In [9]:
input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=dataset_path, 
    desired_s3_uri=s3_dataset_path,
)
print(input_data_uri)

s3://sagemaker-ap-northeast-2-057716757052/fraud2scratch/input/dataset.csv


## 3.변수 저장
---

In [10]:
%store project_prefix
%store base_preproc_input_dir
%store dataset_path 
%store s3_dataset_path 
%store input_data_uri
%store default_bucket



Stored 'project_prefix' (str)
Stored 'base_preproc_input_dir' (str)
Stored 'dataset_path' (str)
Stored 's3_dataset_path' (str)
Stored 'input_data_uri' (str)
Stored 'default_bucket' (str)
